In [ ]:
%matplotlib inline
import openpathsampling as paths
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
flexible = paths.AnalysisStorage("alanine_dipeptide_tps.nc", "r")

In [ ]:
flex_scheme = flexible.schemes[0]

In [ ]:
flex_scheme.move_summary(flexible)

In [ ]:
# TODO: move tree (partial)
import openpathsampling.visualize as visualizations
from IPython.display import SVG

history = visualizations.ReplicaHistoryTree(
    storage=flexible,
    steps=flexible.steps[0:50],
    replica=0
)

In [ ]:
history.view.zoom = 1.0
history.view.scale_y = 24
history.view.scale_x = 10
history.view.font_size = 0.35
history.view.horizontal_gap = 0.05

SVG(history.view.to_svg())

In [ ]:
# TODO: path length histogram
path_lengths = [len(step.active[0].trajectory) for step in flexible.steps]
plt.hist(path_lengths);

In [ ]:
# TODO: path density plots (arbitrary dimension!)

In [ ]:
# TODO: "transition state analysis"
# my thought: 
# * pick 100 (decorrelated) trajectories
# * take every 3rd frame (or something like that)
# * run 50 (or 100?) committor shots with randomized initial velocities
# * say something about which points are likely at/near the 50% isocommittor